# Установка и импорт библиотек

In [ ]:
!pip install haversine
!pip install catboost;
!pip install ipywidgets;
!jupyter nbextension enable --py widgetsnbextension;

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
from google.colab import files
import pandas as pd
import numpy as np
import random
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

import catboost
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Добавление открытых данных о метро

На мой взгляд, метро оказывает сильное воздействие на недвижимость.

In [ ]:
df_1 = pd.read_excel('/content/drive/My Drive/HackTheRealty/E/data/metro.xlsx')
df = df[['NameOfStation_en', 'Longitude_WGS84_en', 'Latitude_WGS84_en']]
df = df.reset_index().drop('index', axis=1)
df = df.rename({'NameOfStation_en': 'metro_station', 'Longitude_WGS84_en': 'metro_longitude', 'Latitude_WGS84_en': 'metro_latitude'}, axis=1)

sr = df_1['NameOfStation_en']
sr[df_1['Name_en'] == 'Коммунарка, вход-выход 4'] = 'Kommunarka'
sr[df_1['Name_en'] == 'Ольховая, вход-выход 1'] = 'Olkhovaya'
sr[df_1['Name_en'] == 'Прокшино, вход-выход 2'] = 'Prokshino'
sr[df_1['Name_en'] == 'Филатов луг, вход-выход 2'] = 'Filatov lug'
sr[df_1['Name_en'] == 'Савёловская, вход-выход 2'] = 'Savelovskaya'
sr[df_1['Name_en'] == 'Коптево, вход-выход 2'] = 'Koptevo'
sr[df_1['Name_en'] == 'Панфиловская, вход-выход 1'] = 'Panfilovskaya'
sr[df_1['Name_en'] == 'Зорге, вход-выход 3'] = 'Zorge'
sr[df_1['Name_en'] == 'Беломорская, вход-выход 2'] = 'Belomorskaya'
sr[df_1['Name_en'] == 'Говорово, вход-выход 1 в вестибюль 2'] = 'Govorovo'
sr[df_1['Name_en'] == 'Озерная, вход-выход 1, вестибюль 2'] = 'Ozernaya'
sr[df_1['Name_en'] == 'Шелепиха, вход-выход 4 во 2-й вестибюль'] = 'Shelepiha'
sr[df_1['Name_en'] == 'ЦСКА, вход-выход 1 во 2-й вестибюль'] = 'CSKA'
sr[df_1['Name_en'] == 'Петровский парк, вход-выход 3 во 2-й вестибюль'] = 'Petrovskii park' 

df_1['NameOfStation_en'] = sr

df_1 = df_1.drop_duplicates(subset=['NameOfStation_en'], ignore_index=True)
df = df_1[['NameOfStation_en', 'Longitude_WGS84_en', 'Latitude_WGS84_en']]
df = df.rename({'NameOfStation_en': 'metro_station', 'Longitude_WGS84_en': 'metro_longitude', 'Latitude_WGS84_en': 'metro_latitude'}, axis=1)

# df.to_csv('metro.csv', index=False)
# files.download("metro.csv")

# Preproccessing 

In [ ]:
def data_preproccesing(train_data):

  train_data['renovation_int'] = train_data['renovation'].map({ 'UNKNOWN': 0, 
                                                      'COSMETIC_DONE': 1, 
                                                      'EURO': 2, 
                                                      'GOOD': 3,
                                                      'NORMAL': 4,
                                                      'DESIGNER_RENOVATION': 5,
                                                      'NEEDS_RENOVATION': 6,
                                                      'RENOVATED': 7,
                                                      'PARTIAL_RENOVATION': 8,
                                                      'COSMETIC_REQUIRED': 9,
                                                      'PRIME_RENOVATION': 10}) 
  train_data = train_data.drop(['renovation'], axis=1)
  train_data = train_data.rename({'renovation_int': 'renovation'}, axis=1)

  train_data['balcony_int'] = train_data['balcony'].map({ 'UNKNOWN': 0, 
                                                      'BALCONY': 1, 
                                                      'LOGGIA': 2, 
                                                      'TWO_LOGGIA': 3,
                                                      'TWO_BALCONY': 4,
                                                      'BALCONY__LOGGIA': 5,
                                                      'BALCONY__TWO_LOGGIA': 6,
                                                      'THREE_LOGGIA': 7,
                                                      'THREE_BALCONY': 8 }) 
  train_data = train_data.drop(['balcony'], axis=1)
  train_data = train_data.rename({'balcony_int': 'balcony'}, axis=1)

  train_data['building_type_int'] = train_data['building_type'].map({ 'PANEL': 0, 
                                                      'BRICK': 1, 
                                                      'MONOLIT': 2, 
                                                      'UNKNOWN': 3,
                                                      'BLOCK': 4,
                                                      'MONOLIT_BRICK': 5,
                                                      'WOOD': 6 }) 
  train_data = train_data.drop(['building_type'], axis=1)
  train_data = train_data.rename({'building_type_int': 'building_type'}, axis=1)

  train_data['parking_int'] = train_data['parking'].map({ 'UNKNOWN': 0, 
                                                      'OPEN': 1, 
                                                      'UNDERGROUND': 2, 
                                                      'CLOSED': 3 }) 
  train_data = train_data.drop(['parking'], axis=1)
  train_data = train_data.rename({'parking_int': 'parking'}, axis=1)

  train_data['day'] = pd.to_datetime(train_data['day'])
  train_data['timestamp'] = train_data['day'].values.astype(np.int64) // 10 ** 9
  train_data = train_data.drop(['day'], axis=1)

  train_data = train_data.set_index(['id'])
  train_data['timestamp'] = train_data['timestamp'].astype(float)
  train_data['price'] = train_data['price'].astype(float)
  train_data['floor'] = train_data['floor'].astype(float)

  train_data = train_data.drop(['studio', 'has_elevator', 'expect_demolition', 'is_apartment'], axis=1)
  train_data = train_data.drop(['site_id'], axis=1)
  train_data = train_data.drop(['total_area'], axis=1)
  train_data = train_data.drop(['flats_count'], axis=1)
  train_data = train_data.drop(['building_id'], axis=1)
  train_data = train_data.drop(['floors_total'], axis=1)
  train_data = train_data.drop(['kitchen_area'], axis=1)

  train_data['build_year'] = train_data['build_year'].astype(float)
  train_data = train_data.drop(['unified_address'], axis=1) 

  le = preprocessing.LabelEncoder()
  train_data['locality_name'] = le.fit_transform(train_data['locality_name'].values)

  ser = pd.Series(train_data['main_image'])
  ser[ser.isnull()] = 0
  ser[ser != 0] = 1
  ser = ser.astype(int)
  train_data['main_image'] = ser

  return train_data

In [ ]:
train_data = pd.read_csv('/content/drive/My Drive/HackTheRealty/E/data/exposition_train.tsv', sep='\t')
test_data = pd.read_csv('/content/drive/My Drive/HackTheRealty/E/data/exposition_test.tsv', sep='\t')

In [ ]:
train_df = data_preproccesing(train_data)
train_df = train_df.drop(['target_string'], axis=1)

test_df = data_preproccesing(test_data)
arr = []
for name in train_df:
  if name != 'target':
    arr.append(name)
test_df = test_df[arr]

In [ ]:
metro = pd.read_csv('/content/drive/My Drive/HackTheRealty/E/data/metro.csv')

def nearest_metro(house, df):
  min_dist = 100
  near_metro = ''
  for i in range(len(df['metro_station'])):
    metro = (df['metro_latitude'][i], df['metro_longitude'][i])
    if haversine(house, metro) < min_dist:
      min_dist = haversine(house, metro)
      near_metro = df['metro_station'][i]
  return min_dist

In [ ]:
metro_distances_test = []

lat_test = np.array(test_df['latitude'])
lon_test = np.array(test_df['longitude'])

for i in tqdm_notebook(range(len(lat_test))):
  house = (lat_test[i], lon_test[i])
  metro_distances_test.append(nearest_metro(house, metro))

metro_distances = []

lat = np.array(train_df['latitude'])
lon = np.array(train_df['longitude'])

for i in tqdm_notebook(range(len(lat))):
  house = (lat[i], lon[i])
  metro_distances.append(nearest_metro(house, metro))

In [ ]:
test_df['metro'] = metro_distances_test
train_df['metro'] = metro_distances

In [ ]:
# test_df.to_csv('test_df.csv', index=False)
# files.download("test_df.csv")

# train_df.to_csv('train_df.csv', index=False)
# files.download("train_df.csv")

In [ ]:
train_df['timestamp'] = train_df['timestamp'] / 10e9
test_df['timestamp'] = test_df['timestamp'] / 10e9

In [ ]:
train_df['is_living'] = 0
train_df['is_living'][train_df['living_area'] != 0] = 1
train_df = train_df.drop('living_area', axis=1)

test_df['is_living'] = 0
test_df['is_living'][test_df['living_area'] != 0] = 1
test_df = test_df.drop(['living_area'], axis=1)

# Training CatBoostRegressor

In [ ]:
X = train_df.drop('target', axis=1)
y = train_df['target']

categorical_features_indices = np.where(X.dtypes != np.float)[0]

categorical_features_indices

In [ ]:
modelR = CatBoostRegressor(iterations=1000,
                             learning_rate=0.02,
                             depth=10,
                             eval_metric='RMSE',
                             random_seed = 1717,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             od_wait=100)

In [ ]:
modelR.fit(X, y, cat_features=categorical_features_indices)

In [ ]:
# modelR.save_model('modelR')
# files.download('modelR')

# Prediction

In [ ]:
pred = np.around(modelR.predict(test_df)).astype(int)

In [ ]:
# submission = pd.read_csv('/content/drive/My Drive/HackTheRealty/E/data/exposition_sample_submission.tsv', sep='\t')
# submission['target'] = pred
# submission.to_csv('submission_last.tsv', sep='\t', index=False)
# files.download("submission_last.tsv")